In [11]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.contrib.explain import GraphMaskExplainer
from torch_geometric.datasets import Planetoid
from torch_geometric.explain import Explainer
from torch_geometric.nn import GATConv, GCNConv
from torch_geometric.explain.metric import fidelity,characterization_score,unfaithfulness
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path = 'data\Planetoid'
dataset = Planetoid(path, name='Cora')
data = dataset[0].to(device)

In [12]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


In [13]:
topk = 40
node_index=10

explainer = Explainer(
    model=model,
    algorithm=GraphMaskExplainer(2, epochs=5),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
threshold_config=dict(threshold_type  = 'topk', value=topk)
)

In [14]:
node_index = 10
explanation = explainer(data.x, data.edge_index, index=node_index)
print(f'Generated explanations in {explanation.available_explanations}')

Train explainer for node(s) 10 with layer 1:   0%|          | 0/5 [00:00<?, ?it/s]

Explain: 100%|██████████| 2/2 [00:00<00:00, 95.28it/s] 

Generated explanations in ['edge_mask', 'node_mask']


In [15]:
unfaithfulness1=unfaithfulness(explainer,explanation,top_k=10)
print(unfaithfulness1)

0.6571427285671234


In [18]:
unfaithfulness1=unfaithfulness(explainer,explanation,top_k=10)
print(unfaithfulness1)

0.6571427285671234


In [19]:
explanation_metrics = explainer(data.x, data.edge_index)

fid_pm = fidelity(explainer, explanation_metrics)
print("Fidelity:", fid_pm)

char_score = characterization_score(fid_pm[0], fid_pm[1])
print("Characterization score:", char_score)

Explain: 100%|██████████| 2/2 [00:00<00:00, 111.09it/s]

Generated explanations in ['edge_mask', 'node_mask']
Fidelity: (0.0697932243347168, 0.07348597049713135)
Characterization score: 0.12980815157428835
